## Environments

In [7]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/FIT5217 Assignment2/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load DataSet

In [6]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def normalizeString(s):
    s = ''.join(
        c for c in unicodedata.normalize('NFD', str(s))
        if unicodedata.category(c) != 'Mn'
    ).lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

MAX_LENGTH = 100

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData():
    train_df = pd.read_csv(file_path + 'train.csv')
    dev_df = pd.read_csv(file_path + 'dev.csv')
    test_df = pd.read_csv(file_path + 'test.csv')

    train_pairs = [[normalizeString(row['Ingredients']), normalizeString(row['Recipe'])] for index, row in train_df.iterrows()]
    dev_pairs = [[normalizeString(row['Ingredients']), normalizeString(row['Recipe'])] for index, row in dev_df.iterrows()]
    test_pairs = [[normalizeString(row['Ingredients']), normalizeString(row['Recipe'])] for index, row in test_df.iterrows()]

    print("Read %s training pairs" % len(train_pairs))
    train_pairs = filterPairs(train_pairs)
    print("Trimmed to %s training pairs (sequences shorter than %d tokens)" % (len(train_pairs), MAX_LENGTH))

    input_lang = Lang('ingredients')
    output_lang = Lang('recipe')

    print("\nBuilding vocabulary from training data...")
    for pair in train_pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words:")
    print(f"- {input_lang.name}: {input_lang.n_words} words")
    print(f"- {output_lang.name}: {output_lang.n_words} words")

    return input_lang, output_lang, train_pairs, dev_pairs, test_pairs

input_lang, output_lang, train_pairs, dev_pairs, test_pairs = prepareData()

# Print a random training pair to verify
print("\nExample training pair:")
print(random.choice(train_pairs))

Read 162899 training pairs
Trimmed to 158251 training pairs (sequences shorter than 100 tokens)

Building vocabulary from training data...
Counted words:
- ingredients: 8532 words
- recipe: 12543 words

Example training pair:
[' c . butter or margarine c . powdered sugar c . milk tsp . vanilla use clear vanilla for a white frosting ', ' put butter in bowl and beat with mixed until fluffy . add cups of the powdered sugar then beat in the c . milk and vanilla . add remaining sugar and mix until smooth and fluffy . ']


## Model

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        '''
        GRU is a gated RNN variant that captures long-term dependencies more effectively with fewer parameters. GRU has the same output shape as a standard RNN when configured identically.
        '''
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Prepare Train Data

In [9]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## Train the Model

In [10]:
MAX_LENGTH = 100 # Maximum sentence length to consider

teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

## Show Plot Results

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluation

In [12]:
# This function runs the trained encoder and decoder on a given input sentence to generate an output sequence.
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    # We don't need gradients during evaluation, so we disable them for efficiency
    with torch.no_grad():
        # Convert the input sentence (a string) into a tensor of token indices
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]

        # Initialize the encoder's hidden state
        encoder_hidden = encoder.initHidden()

        # Create a tensor to store encoder outputs at each time step (for attention, if used)
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        # Pass each token of the input through the encoder one at a time
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]  # Store encoder output

        # Decoder starts with the <SOS> (start-of-sequence) token
        decoder_input = torch.tensor([[SOS_token]], device=device)

        # Decoder's initial hidden state is the encoder's final hidden state
        decoder_hidden = encoder_hidden

        # List to store the output words predicted by the decoder
        decoded_words = []

        # Tensor to store attention weights (not used here, but often included for attention visualisation)
        decoder_attentions = torch.zeros(max_length, max_length)

        # Generate words one by one until we reach max_length or predict <EOS>
        for di in range(max_length):
            # Pass current input and hidden state to the decoder
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            # Get the most likely next word (highest probability)
            topv, topi = decoder_output.data.topk(1)

            # If decoder predicts <EOS>, stop decoding
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                # Convert predicted index to actual word and append to output sequence
                decoded_words.append(output_lang.index2word[topi.item()])

            # Use the predicted word as the next input to the decoder (greedy decoding)
            decoder_input = topi.squeeze().detach()

        # Return the list of predicted words as the final output
        return decoded_words

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words= evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Train and Evaluation

In [13]:
# Set the size of the hidden layers in both the encoder and decoder RNNs
hidden_size = 256

# Create an instance of the encoder RNN
# It takes input_lang.n_words as the input vocabulary size (i.e., number of unique input tokens)
# and hidden_size as the size of the RNN's hidden state
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)

# Create an instance of the decoder RNN
# It takes hidden_size from the encoder and output_lang.n_words as the output vocabulary size
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

# Set the total number of training iterations (i.e., how many sentence pairs to train on)
n_iters = 20000

# Set how often to print the training loss
print_every = 2000

plot_losses = trainIters(encoder1, decoder1, n_iters=n_iters, print_every=print_every)

2m 25s (- 21m 46s) (2000 10%) 9.2339
4m 44s (- 18m 58s) (4000 20%) 12.3760
7m 5s (- 16m 32s) (6000 30%) 13.6873
9m 26s (- 14m 9s) (8000 40%) 13.8575
11m 46s (- 11m 46s) (10000 50%) 13.2346
14m 4s (- 9m 23s) (12000 60%) 12.7773
16m 29s (- 7m 4s) (14000 70%) 13.7940
18m 52s (- 4m 43s) (16000 80%) 13.0565
21m 11s (- 2m 21s) (18000 90%) 12.5538
23m 30s (- 0m 0s) (20000 100%) 11.4739
